# Inferring loads on an Euler Bernoulli beam

Here we look at another linear Gaussian inverse problem, but one with a distributed parameter field and PDE forward model.  More specifically, the inverse problem is to infer the forces acting along a cantilevered beam.

<div><img src="CantileverBeam.png" width="500px" style="padding-bottom:0.5em;"/><br>Image courtesy of wikimedia.org</div>
  


### Formulation:

Let $u(x)$ denote the vertical deflection of the beam and let $m(x)$ denote the vertial force acting on the beam at point $x$ (positive for upwards, negative for downwards).  We assume that the displacement can be well approximated using Euler-Bernoulli beam theory and thus satisfies the PDE
$$
\frac{\partial^2}{\partial x^2}\left[ E(x) \frac{\partial^2 u}{\partial x^2}\right] = m(x),
$$
where $E(x)$ is an effective stiffness that depends both on the beam geometry and material properties.  In this lab, we assume $E(x)$ is constant and $E(x)=10^5$.  However, in future labs, $E(x)$ will also be an inference target. 

For a beam of length $L$, the cantilever boundary conditions take the form
$$
u(x=0) = 0,\quad \left.\frac{\partial u}{\partial x}\right|_{x=0} = 0
$$
and
$$
\left.\frac{\partial^2 u}{\partial x^2}\right|_{x=L} = 0, \quad  \left.\frac{\partial^3 u}{\partial x^3}\right|_{x=L} = 0.
$$
  
Discretizing this PDE with finite differences (or finite elements, etc...), we obtain a linear system of the form
$$
Ku = m,
$$
where $u\in\mathbb{R}^N$ and $m\in\mathbb{R}^N$ are vectors containing approximations of $u(x)$ and $m(x)$ at the finite difference nodes.  

Only $M$ components of the $N$ dimensional vector $u$ are observed.  To account for this, we introduce a sparse matrix $B$ that extracts $M$ unique components of $u$.  Combining this with an additive Gaussian noise model, we obtain  
$$
y = BK^{-1} m + \epsilon,
$$
where $\epsilon \sim N(0,\Sigma_y)$.



## Imports

In [1]:
import sys
sys.path.insert(0,'/home/fenics/Installations/MUQ_INSTALL/lib')

from IPython.display import Image

import numpy as np
import matplotlib.pyplot as plt
import h5py

# MUQ Includes
import pymuqModeling as mm # Needed for Gaussian distribution
import pymuqApproximation as ma # Needed for Gaussian processes

## Read discretization and observations

In [2]:
f = h5py.File('ProblemDefinition.h5','r')
x = np.array( f['/ForwardModel/NodeLocations'] )
K = np.array( f['/ForwardModel/SystemMatrix'] )
B = np.array( f['/Observations/ObservationMatrix'] )
obsData = np.array( f['/Observations/ObservationData'] )

numObs = obsData.shape[0]
numPts = x.shape[1]
dim = 1

## Construct components of the likelihood

In [3]:
obsMat = np.linalg.solve(K,B.T).T
noiseCov = 1e-12 * np.eye(numObs)

## Define a Gaussian Process prior
This is known apriori:
- The true prior variance is 101
- The loads are smooth with a constant offset.
- The load is continuously differentiable

Reference the Gaussian process notebook in this folder for syntax questions.

In [4]:
# TODO: Construct the Gaussian process prior here.

### Plot prior samples

In [5]:
# TODO: Plot several realizations of the prior at the finite difference nodes

### Compute posterior
A Gaussian Process defines a distribution over functions.  The Gaussian process class has a function `Discretize(x)` that will return a Gaussian distribution by evaluating the mean function and covariance kernel at the points contained in `x`. 

For usage of the `Gaussian` class and the `Condition` function, refer to the earlier linear regression notebook.

In [6]:
# TODO: Discretize the GP prior and condition the resulting Gaussian on the observations

### Plot posterior
Look at the posterior spatially.  You might want to consider the mean, marginal standard deviation, or samples of the posterior.

It might help to look at the documentation of matplotlib's `fill_between` function, available [here](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.fill_between.html).

In [7]:
# TODO: Plot the posterior.  

## Incorporate observation of load sum
Assume you are now given an additional observation: the sum of the load vector $m$.
Update the posterior distribution to reflect this new information.

In [8]:
sumObs = np.array( f['/Observations/LoadSum'] )
sumNoiseVar = 1e-7 * np.ones(1)

In [9]:
# TODO: Construct an observation matrix and update the posterior

In [10]:
# TODO: Plot the updated posterior